In [ ]:
from PIL import Image
import os
import csv
#import exifread
from PIL.ExifTags import TAGS
from exif import Image

In [ ]:
master_directory = ""
csv_file = "output.csv"

csv_header = ["filename", "altitude", "gsd"]

with open(csv_file, mode='w', newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    csv_writer.writerow(csv_header)

In [ ]:
# Open an example image and extract EXIF data for altitude (reference for getting any EXIF tag)
image_path = ""

with open(image_path, 'rb') as src:
    img = Image(src)
    #print(img.list_all()) # this is where you can get all the EXIF tag names to get relevant values
    altitude = img.get("gps_altitude")
    print(altitude)

In [ ]:
# Get altitude and GSD for all images-- one site at a time, because ground elevation must be constant

ground_elevation = 1844

# Define sensor parameters (in millimeters)-- this is Mavic 2 Pro
image_width = 5472  # Example image width in pixels
image_height = 3648  # Example image height in pixels
sensor_width = 0.0128  # Make sure this is converted to meters!
sensor_height = 0.0096  # Make sure this is converted to meters!
focal_length = 0.01035  # Make sure this is converted to meters!

altitude_data = []  # List to store extracted altitude values

for transect_id in os.listdir(master_directory):
    transect_dir = os.path.join(master_directory, transect_id)
    
    if os.path.isdir(transect_dir):
        for image_filename in os.listdir(transect_dir):
            if image_filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(transect_dir, image_filename)
                    
                # Open the image and extract EXIF data
                with open(image_path, 'rb') as src:
                    img = Image(src)
    
                    altitude = img.get("gps_altitude")
                # Calculate altitude above ground level
                agl_altitude = altitude - ground_elevation if altitude is not None else None
                altitude_data.append((image_filename, agl_altitude))

# Calculate GSD and write data to CSV
with open(csv_file, mode='a', newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    
    for image_filename, transect_id, agl_altitude in altitude_data:
        if agl_altitude is not None:
            # Calculate ground sampling distance (in meters/px)
            gsd_m = (sensor_width * agl_altitude) / (focal_length * image_width)
            gsd = gsd_m*100 #convert to cm-- optional
        else:
            gsd = None

        # Write data to CSV
        csv_writer.writerow([image_filename, agl_altitude, gsd])


In [ ]:
#Using a csv output from ExifTool that contains Relative Altitude (DJI drones only, so far)
import csv
# Define sensor parameters (in millimeters)-- this is Mavic 2 Pro
image_width = 5472  # Example image width in pixels
image_height = 3648  # Example image height in pixels
sensor_width = 0.0128  # Make sure this is converted to meters!
sensor_height = 0.0096  # Make sure this is converted to meters!
focal_length = 0.01035  # Make sure this is converted to meters!

altitude_data = []  # List to store extracted altitude values

#Requires CSV output from ExifTool
with open('exif.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    for row in csv_reader:
        # Extract the "RelativeAltitude" and "FileName" values
        relative_altitude = row.get("RelativeAltitude", None)
        image_filename = row.get("FileName", None)

        if relative_altitude is not None and relative_altitude.strip() != '' and image_filename is not None:
            try:
                relative_altitude = float(relative_altitude)
                if relative_altitude > 120:
                    # Flag values exceeding 120 as likely absolute altitudes
                    print(f"Skipped: {image_filename} - RelativeAltitude exceeds legal flight ceiling: {relative_altitude}")
                else:
                    altitude_data.append((image_filename, relative_altitude, None))
            except ValueError:
                print(f"Skipping: {image_filename} - Invalid RelativeAltitude value: {relative_altitude}")

# Calculate GSD and write data to a new CSV file
with open('output.csv', mode='w', newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    csv_writer.writerow(["Image Filename", "Relative Altitude (m)", "GSD (cm)"])
    
    for image_filename, relative_altitude in altitude_data:
        # Calculate GSD (in cm)
        gsd_m = (sensor_width * relative_altitude) / (focal_length * image_width)
        gsd_cm = gsd_m * 100  # Convert to cm

        # Write data to the new CSV file
        csv_writer.writerow([image_filename, relative_altitude, gsd_cm])

In [ ]:
#Hacked together fix for one site with improperly specified relative altitude values
import csv

# Define sensor parameters (in meters) - Mavic 2 Pro
image_width = 5472  # Example image width in pixels
image_height = 3648  # Example image height in pixels
sensor_width = 0.0128  # Make sure this is converted to meters!
sensor_height = 0.0096  # Make sure this is converted to meters!
focal_length = 0.01035  # Make sure this is converted to meters!

# Requires CSV input from ExifTool
altitude_data = []
with open('exif.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    for row in csv_reader:
        # Extract the "RelativeAltitude" and "FileName" values
        relative_altitude = row.get("RelativeAltitude", None)
        image_filename = row.get("FileName", None)

        if relative_altitude is not None and relative_altitude.strip() != '' and image_filename is not None:
            try:
                relative_altitude = float(relative_altitude)
                if relative_altitude > 120:
                    # Flag values exceeding 120 as likely absolute altitudes
                    altitude_data.append((image_filename, None, relative_altitude))
                else:
                    altitude_data.append((image_filename, relative_altitude, None))
            except ValueError:
                print(f"Skipping row: {image_filename} - Invalid RelativeAltitude value: {relative_altitude}")

# Calculate GSD and write data to a new CSV file
with open('output.csv', mode='w', newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    csv_writer.writerow(["Image Filename", "Relative Altitude (m)", "GSD (cm)"])
    
    for image_filename, relative_altitude, absolute_altitude in altitude_data:
        
        # Calculate GSD (in m)
        if relative_altitude is not None:
            gsd_m = (sensor_width * relative_altitude) / (focal_length * image_width)
        else:
            ground_elevation = 1278  # Specify the ground elevation value
            relative_altitude = absolute_altitude - ground_elevation
            gsd_m = (sensor_width * relative_altitude) / (focal_length * image_width)

        gsd_cm = gsd_m * 100 if gsd_m is not None else None  # Convert to cm

        # Write data to the new CSV file
        csv_writer.writerow([image_filename, relative_altitude, gsd_cm])